# Getting Raw Penalty Data from Roboflow

In [ ]:
import os
from roboflow import Roboflow

rf = Roboflow(api_key="iRQrxHG4sWzHjwqwu44a")
project = rf.workspace("footballai-xndiy").project("drohnenvideos")
version = project.version(3)
dataset = version.download("yolov12")

# Speicherort: Ein Ordner über dem aktuellen Verzeichnis
parent_dir = os.path.abspath(os.path.join(os.getcwd()))  # Geht eine Ebene nach oben
dataset_path = os.path.join(parent_dir, "raw_images")  # Neuer Speicherort

# Dataset herunterladen in den übergeordneten Ordner
dataset = version.download("yolov12", location=dataset_path)

print(f"✅ Dataset gespeichert in: {dataset.location}")     

                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Drohnenvideos-3 in yolov12:: 100%|██████████| 4464/4464 [00:03<00:00, 1435.68it/s]


Extracting Dataset Version Zip to c:\Users\hause\OneDrive\01_Studium\02_Master\05_Masterarbeit\drohnenvideos-yolov12 in yolov12:: 100%|██████████| 4464/4464 [00:03<00:00, 1246.22it/s]

✅ Dataset gespeichert in: c:\Users\hause\OneDrive\01_Studium\02_Master\05_Masterarbeit\drohnenvideos-yolov12


# Train/Test/Val Splitting

A Split onto the raw 38 images is done using a 55/30/15 split

In [ ]:
import os
import cv2
import random
from tqdm import tqdm

# Originale Bilder und Labels
IMG_DIR = "raw_images/images"
LABEL_DIR = "raw_images/labels"

# Zielverzeichnisse für Split
BASE_OUT = "cropped"
SPLITS = {
    "train": 0.55,
    "val": 0.30,
    "test": 0.15
}

# Zielordner anlegen
for split in SPLITS.keys():
    os.makedirs(os.path.join(BASE_OUT, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(BASE_OUT, split, "labels"), exist_ok=True)

# Cropping Funktion (Ziel: Tor mittig im Crop)
def crop_around_target_center(image, crop_size=640, target_x_center_ratio=0.7, target_y_center_ratio=0.35):
    h, w = image.shape[:2]
    center_x = int(target_x_center_ratio * w)
    center_y = int(target_y_center_ratio * h)
    x1 = max(0, min(center_x - crop_size // 2, w - crop_size))
    y1 = max(0, min(center_y - crop_size // 2, h - crop_size))
    crop = image[y1:y1+crop_size, x1:x1+crop_size]
    return crop, x1, y1

# Bounding Box anpassen & ungültige entfernen
def adjust_boxes_to_crop(boxes, class_labels, x_crop, y_crop, orig_w, orig_h, crop_size=640):
    new_boxes = []
    new_labels = []

    for label, box in zip(class_labels, boxes):
        x_c, y_c, bw, bh = box
        abs_x = x_c * orig_w
        abs_y = y_c * orig_h
        abs_w = bw * orig_w
        abs_h = bh * orig_h

        new_abs_x = abs_x - x_crop
        new_abs_y = abs_y - y_crop
        new_x = new_abs_x / crop_size
        new_y = new_abs_y / crop_size
        new_w = abs_w / crop_size
        new_h = abs_h / crop_size

        if any(val < 0 or val > 1 for val in [new_x, new_y, new_w, new_h]):
            continue

        new_boxes.append([new_x, new_y, new_w, new_h])
        new_labels.append(label)

    return new_boxes, new_labels

# Helper: Labels laden und speichern
def load_label(label_path):
    boxes = []
    class_labels = []
    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) == 5:
                class_labels.append(parts[0])
                boxes.append([float(p) for p in parts[1:]])
    return boxes, class_labels

def save_label(label_path, boxes, class_labels):
    with open(label_path, 'w') as f:
        for label, box in zip(class_labels, boxes):
            f.write(f"{label} {' '.join(f'{x:.6f}' for x in box)}\n")

# MAIN
image_files = [f for f in os.listdir(IMG_DIR) if f.endswith((".jpg", ".jpeg", ".png"))]
random.shuffle(image_files)

n_total = len(image_files)
n_val = int(n_total * SPLITS["val"])
n_test = int(n_total * SPLITS["test"])

val_files = image_files[:n_val]
test_files = image_files[n_val:n_val+n_test]
train_files = image_files[n_val+n_test:]

split_map = {
    **{f: "val" for f in val_files},
    **{f: "test" for f in test_files},
    **{f: "train" for f in train_files},
}

for image_file in tqdm(image_files):
    image_path = os.path.join(IMG_DIR, image_file)
    label_path = os.path.join(LABEL_DIR, image_file.rsplit('.', 1)[0] + ".txt")

    if not os.path.exists(label_path):
        continue

    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    boxes, class_labels = load_label(label_path)

    crop_img, x1, y1 = crop_around_target_center(image)
    crop_boxes, crop_labels = adjust_boxes_to_crop(boxes, class_labels, x1, y1, w, h)

    if not crop_boxes:
        continue  # überspringen wenn keine gültigen Boxen

    split = split_map[image_file]
    out_img_path = os.path.join(BASE_OUT, split, "images", image_file)
    out_label_path = os.path.join(BASE_OUT, split, "labels", image_file.rsplit('.', 1)[0] + ".txt")

    cv2.imwrite(out_img_path, crop_img)
    save_label(out_label_path, crop_boxes, crop_labels)


100%|██████████| 38/38 [00:00<00:00, 40.15it/s]


# Albumentation

- First the images get cropped so that the players and goal box is centered in the image and has the dimensions of 640px that YOLO requires

- Secondly, the images get augmented to create 208 training images

In [ ]:
import os
import cv2
import random
import albumentations as A
from tqdm import tqdm

# Verzeichnisse
IMG_DIR = "cropped/images"
LABEL_DIR = "cropped/labels"
OUT_IMG_DIR = "train_augmented/images"
OUT_LABEL_DIR = "train_augmented/labels"

os.makedirs(OUT_IMG_DIR, exist_ok=True)
os.makedirs(OUT_LABEL_DIR, exist_ok=True)

# -----------------------------
# Cropping: Rechts & Mittig
# -----------------------------
def crop_right_center(image, 
    crop_size=640, 
    target_x_center_ratio=0.7, 
    target_y_center_ratio=0.35  # Standard: vertikal mittig
):
    h, w = image.shape[:2]

    # Berechne Zielmittelpunkt in Pixeln
    center_x = int(target_x_center_ratio * w)
    center_y = int(target_y_center_ratio * h)

    # Berechne linke obere Ecke für Crop
    x1 = center_x - crop_size // 2
    y1 = center_y - crop_size // 2

    # Begrenzung, damit Crop im Bild bleibt
    x1 = max(0, min(x1, w - crop_size))
    y1 = max(0, min(y1, h - crop_size))

    crop = image[y1:y1+crop_size, x1:x1+crop_size]
    return crop, x1, y1

def adjust_boxes_to_crop(boxes, class_labels, x_crop, y_crop, orig_w, orig_h, crop_size=640):
    new_boxes = []
    new_labels = []

    for label, box in zip(class_labels, boxes):
        x_c, y_c, bw, bh = box

        # Absolute Werte im Originalbild
        abs_x = x_c * orig_w
        abs_y = y_c * orig_h
        abs_w = bw * orig_w
        abs_h = bh * orig_h

        # Neue Position im Crop
        new_abs_x = abs_x - x_crop
        new_abs_y = abs_y - y_crop
        new_x = new_abs_x / crop_size
        new_y = new_abs_y / crop_size
        new_w = abs_w / crop_size
        new_h = abs_h / crop_size

        # Filter: Werte müssen im Bereich [0, 1] liegen
        if any(val < 0 or val > 1 for val in [new_x, new_y, new_w, new_h]):
            # Debug-Zeile (kannst du auch auskommentieren)
            continue
        else: 
            new_boxes.append([new_x, new_y, new_w, new_h])
            new_labels.append(label)

    return new_boxes, new_labels



# -----------------------------
# Helper: Label laden/speichern
# -----------------------------
def load_label(label_path):
    boxes = []
    class_labels = []
    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) == 5:
                class_labels.append(parts[0])
                boxes.append([float(p) for p in parts[1:]])
    return boxes, class_labels

def save_label(label_path, boxes, class_labels):
    with open(label_path, 'w') as f:
        for label, box in zip(class_labels, boxes):
            f.write(f"{label} {' '.join(f'{x:.6f}' for x in box)}\n")


# -----------------------------
# AUGMENTIERUNG: Realistisch
# -----------------------------
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5), 
    A.ToGray(p=0.3),
    A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=5, p=0.3),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))


# -----------------------------
# MAIN AUGMENT LOOP
# -----------------------------
image_files = [f for f in os.listdir(IMG_DIR) if f.endswith((".jpg", ".png", ".jpeg"))]

for image_file in tqdm(image_files):
    image_path = os.path.join(IMG_DIR, image_file)
    label_path = os.path.join(LABEL_DIR, image_file.rsplit('.', 1)[0] + ".txt")

    if not os.path.exists(label_path):
        continue

    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    boxes, class_labels = load_label(label_path)

    for i in range(9):
        # Cropping
        cropped_img, x1, y1 = crop_right_center(image)
        cropped_boxes, cropped_labels = adjust_boxes_to_crop(boxes, class_labels, x1, y1, w, h)

        try:
            transformed = transform(image=cropped_img, bboxes=cropped_boxes, class_labels=cropped_labels)
        except ValueError as e:
            print(f"Fehler bei Datei {image_file}: {e}")
            continue
        aug_img = transformed["image"]
        aug_boxes = transformed["bboxes"]
        aug_labels = transformed["class_labels"]

        if not aug_boxes:
            continue  # Keine gültigen BBoxes im Crop

        # Speichern
        base_name = image_file.rsplit('.', 1)[0]
        new_img_name = f"{base_name}_aug{i}.jpg"
        new_label_name = f"{base_name}_aug{i}.txt"

        cv2.imwrite(os.path.join(OUT_IMG_DIR, new_img_name), aug_img)
        save_label(os.path.join(OUT_LABEL_DIR, new_label_name), aug_boxes, aug_labels)


 37%|███▋      | 14/38 [00:01<00:01, 18.46it/s]

Fehler bei Datei frame_0009_jpg.rf.92c126f17b8f12d0ba3ddda922b36d7c.jpg: Expected x_min for bbox [-0.00234375  0.23828126  0.01640625  0.29140627  2.        ] to be in the range [0.0, 1.0], got -0.0023437505587935448.
Fehler bei Datei frame_0009_jpg.rf.92c126f17b8f12d0ba3ddda922b36d7c.jpg: Expected x_min for bbox [-0.00234375  0.23828126  0.01640625  0.29140627  2.        ] to be in the range [0.0, 1.0], got -0.0023437505587935448.
Fehler bei Datei frame_0009_jpg.rf.92c126f17b8f12d0ba3ddda922b36d7c.jpg: Expected x_min for bbox [-0.00234375  0.23828126  0.01640625  0.29140627  2.        ] to be in the range [0.0, 1.0], got -0.0023437505587935448.
Fehler bei Datei frame_0009_jpg.rf.92c126f17b8f12d0ba3ddda922b36d7c.jpg: Expected x_min for bbox [-0.00234375  0.23828126  0.01640625  0.29140627  2.        ] to be in the range [0.0, 1.0], got -0.0023437505587935448.
Fehler bei Datei frame_0009_jpg.rf.92c126f17b8f12d0ba3ddda922b36d7c.jpg: Expected x_min for bbox [-0.00234375  0.23828126  0.016

 53%|█████▎    | 20/38 [00:01<00:00, 27.57it/s]

Fehler bei Datei frame_0020_jpg.rf.22cdc7cde9a5a661a67386eacffbcfb6.jpg: Expected x_min for bbox [-0.00546875  0.39609376  0.01953125  0.46015623  1.        ] to be in the range [0.0, 1.0], got -0.00546875037252903.
Fehler bei Datei frame_0020_jpg.rf.22cdc7cde9a5a661a67386eacffbcfb6.jpg: Expected x_min for bbox [-0.00546875  0.39609376  0.01953125  0.46015623  1.        ] to be in the range [0.0, 1.0], got -0.00546875037252903.
Fehler bei Datei frame_0020_jpg.rf.22cdc7cde9a5a661a67386eacffbcfb6.jpg: Expected x_min for bbox [-0.00546875  0.39609376  0.01953125  0.46015623  1.        ] to be in the range [0.0, 1.0], got -0.00546875037252903.
Fehler bei Datei frame_0020_jpg.rf.22cdc7cde9a5a661a67386eacffbcfb6.jpg: Expected x_min for bbox [-0.00546875  0.39609376  0.01953125  0.46015623  1.        ] to be in the range [0.0, 1.0], got -0.00546875037252903.
Fehler bei Datei frame_0020_jpg.rf.22cdc7cde9a5a661a67386eacffbcfb6.jpg: Expected x_min for bbox [-0.00546875  0.39609376  0.01953125  0

100%|██████████| 38/38 [00:02<00:00, 12.81it/s]
